In [ ]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
from tqdm import tqdm

In [ ]:


df = pd.read_csv("Saudi.csv").sample(n=1600, random_state=42).reset_index(drop=True)
texts = df["tweet"].tolist()

In [ ]:


model_id = "ALLaM-AI/ALLaM-7B-Instruct-preview"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
import random

def create_examples(df, n_per_class=3):
    offensive_examples = df[df["label"].str.lower() == "offensive"].sample(n=n_per_class, random_state=1)
    not_examples = df[df["label"].str.lower() == "not"].sample(n=n_per_class, random_state=1)

    examples = []
    for _, row in pd.concat([offensive_examples, not_examples]).iterrows():
        examples.append({
            "text": row["tweet"],
            "label": row["label"].strip().lower()
        })
    return examples

def create_prompt(system_message, examples, user_template):
    messages = [{"role": "system", "content": system_message}]
    for ex in examples:
        messages.append({"role": "user", "content": user_template.format(text=ex["text"])})
        messages.append({"role": "assistant", "content": ex["label"]})
    return messages



system_message = """
Classify the input Arabic tweet as 'offensive' or 'not'.
The text will be delimited by triple backticks ``` in the input.
Answer only with 'offensive' or 'not'. Do not explain your answer!
"""

user_message_template = "Tweet: ```{text}```"

In [ ]:
zero_shot_preds = []
one_shot_preds = []
three_shot_preds = []

one_example = create_examples(df, n_per_class=1)
three_examples = create_examples(df, n_per_class=3)

model.eval()

import re

for i, text in enumerate(tqdm(texts)):
    # --- Zero-shot
    prompt = f"""
    {system_message}
    {user_message_template.format(text=text)}
    Answer:
    """
    zs_input = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        zs_output = model.generate(**zs_input, max_new_tokens=5)
    label = tokenizer.decode(zs_output[0], skip_special_tokens=True).split("Answer:")[-1].strip().lower()
    zero_shot_preds.append(label)

    # --- One-shot
    one_prompt = create_prompt(system_message, one_example, user_message_template)
    one_prompt.append({"role": "user", "content": user_message_template.format(text=text)})
    one_input = tokenizer.apply_chat_template(one_prompt, tokenize=False, add_generation_prompt=True)
    one_input_ids = tokenizer(one_input, return_tensors="pt").to(model.device)
    with torch.no_grad():
        one_output = model.generate(**one_input_ids, max_new_tokens=50)

    one_decoded = tokenizer.decode(one_output[0], skip_special_tokens=True)
    label = one_decoded.strip().split()[-1].lower()
    one_shot_preds.append(label)

    # --- Few-shot (3-shot)
    few_prompt = create_prompt(system_message, three_examples, user_message_template)
    few_prompt.append({"role": "user", "content": user_message_template.format(text=text)})
    few_input = tokenizer.apply_chat_template(few_prompt, tokenize=False, add_generation_prompt=True)
    few_input_ids = tokenizer(few_input, return_tensors="pt").to(model.device)
    with torch.no_grad():
        few_output = model.generate(**few_input_ids, max_new_tokens=50)

    few_decoded = tokenizer.decode(few_output[0], skip_special_tokens=True)
    label = few_decoded.strip().split()[-1].lower()

    three_shot_preds.append(label)

    time.sleep(0.2)


  0%|          | 0/1600 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/1600 [00:02<1:09:11,  2.60s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 2/1600 [00:03<48:36,  1.82s/it]  Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 3/1600 [00:05<42:48,  1.61s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generati

In [ ]:

df["allam_zero_shot"] = zero_shot_preds
df["allam_one_shot"] = one_shot_preds
df["allam_three_shot"] = three_shot_preds

df.to_csv("allam_results.csv", index=False)
